I'm pretty limited for this figure, because I drew most of it by hand in inkscape, which isn't always the best idea. Nevertheless, I can atleast go through the process of generating the data for that figure.

# Libraries

In [1]:
%load_ext rpy2.ipython

In [2]:
%%R 
library(tidyverse)
library(readxl)
library(corrplot)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: corrplot 0.92 loaded



# Data

In [3]:
%%R
best_hits <- read_csv("data/Ecoli_pangenome_best_hits.csv")

best_hits_hmmer <- best_hits %>%
    filter(hmmerHit == TRUE)

best_hits_hmmer

Rows: 7262 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): qseqid, sseqid, component, cinful_id, pephash, sample, contig, seq
dbl (13): pident, length, mismatch, gapopen, qstart, qend, sstart, send, eva...
lgl  (4): hmmerHit, verified, allStandardAA, signalMatch

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 769 × 25
   qseqid        sseqid pident length mismatch gapopen qstart  qend sstart  send
   <chr>         <chr>   <dbl>  <dbl>    <dbl>   <dbl>  <dbl> <dbl>  <dbl> <dbl>
 1 v1_PLS_987b1… E492_…   80.4     92       18       0      1    92      1    92
 2 v1_PLS_07196… L_tr_…   99.0    102        1       0      1   102      4   105
 3 v1_PLS_718e5… V_sp_…   81.2     16        3       0      1    16      1    16
 4 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
 5 v1

In [4]:
%%R
assemblySummary_ENAInformation <- read_excel("data/Ecoli_assemblySummary.xlsx", sheet="ENA-Information")

In [5]:
%%R -o assemblySummary_SequencedGenomes
assemblySummary_SequencedGenomes <- read_excel("data/Ecoli_assemblySummary.xlsx", sheet="Sequenced-Genomes", skip=5) %>%
#	separate("Sequence_Name",c("phylogroup")) %>%
	select(c("Genome_ID","Isolation source","Strain Category")) %>%
	rename("GEMBASE_NAME"="Genome_ID") %>%
	drop_na()


assemblySummary_SequencedGenomes

New names:
• `` -> `...8`
• `` -> `...20`
• `` -> `...28`
• `` -> `...35`
# A tibble: 1,304 × 3
   GEMBASE_NAME    `Isolation source` `Strain Category`     
   <chr>           <chr>              <chr>                 
 1 ESCO.0216.00064 Host-associated    Mammal_Faecal         
 2 ESCO.0216.00065 Environmental      Water                 
 3 ESCO.0216.00066 Host-associated    Human_Extra-intestinal
 4 ESCO.0216.00067 Host-associated    Human_Extra-intestinal
 5 ESCO.0216.00068 Host-associated    Mammal_Faecal         
 6 ESCO.0216.00069 Host-associated    Human_Extra-intestinal
 7 ESCO.0216.00070 Environmental      Water                 
 8 ESCO.0216.00071 Environmental      Water                 
 9 ESCO.0216.00072 Host-associated    Human_Intestinal      
10 ESCO.0216.00073 Host-associated    Human_Intestinal      
# … with 1,294 more rows


In [6]:
%%R -o assemblySummary_AustralianDataset
assemblySummary_AustralianDataset <- read_excel("data/Ecoli_assemblySummary.xlsx", sheet="Australian-Dataset", skip=4) %>%
	select(c("Genome_ID","Phylogroup")) %>%
	rename("GEMBASE_NAME"="Genome_ID") %>%
	full_join(assemblySummary_SequencedGenomes) %>%
	rename("phylogroup"="Phylogroup") %>%
	drop_na()


assemblySummary_AustralianDataset

New names:
• `` -> `...4`
• `` -> `...13`
• `` -> `...14`
• `` -> `...22`
• `` -> `...32`
• `` -> `...37`
• `` -> `...38`
• `` -> `...39`
• `` -> `...40`
• `` -> `...41`
• `` -> `...42`
• `` -> `...50`
• `` -> `...57`
• `` -> `...65`
• `` -> `...71`
• `` -> `...74`
• `` -> `...75`
• `` -> `...76`
• `` -> `...80`
• `` -> `...83`
• `` -> `...84`
Joining, by = "GEMBASE_NAME"
# A tibble: 1,294 × 4
   GEMBASE_NAME    phylogroup `Isolation source` `Strain Category`     
   <chr>           <chr>      <chr>              <chr>                 
 1 ESCO.0216.00064 A          Host-associated    Mammal_Faecal         
 2 ESCO.0216.00065 B2         Environmental      Water                 
 3 ESCO.0216.00066 B2         Host-associated    Human_Extra-intestinal
 4 ESCO.0216.00067 B2         Host-associated    Human_Extra-intestinal
 5 ESCO.0216.00068 B2         Host-associated    Mammal_Faecal         
 6 ESCO.0216.00069 B2         Host-associated    Human_Extra-intestinal
 7 ESCO.0216.00070 B2      

In [7]:
%%R -o assemblySummary_ENAInformation_phylogroup
assemblySummary_ENAInformation_phylogroup <- assemblySummary_AustralianDataset %>% 
	full_join(assemblySummary_ENAInformation) %>%
	drop_na()

assemblySummary_ENAInformation_phylogroup

Joining, by = "GEMBASE_NAME"
# A tibble: 1,224 × 9
   GEMBASE_NAME    phylogroup `Isolation source` `Strain Category` ASSEMBLY_NAME
   <chr>           <chr>      <chr>              <chr>             <chr>        
 1 ESCO.0216.00064 A          Host-associated    Mammal_Faecal     M919         
 2 ESCO.0216.00065 B2         Environmental      Water             E5795        
 3 ESCO.0216.00066 B2         Host-associated    Human_Extra-inte… H219         
 4 ESCO.0216.00069 B2         Host-associated    Human_Extra-inte… H144         
 5 ESCO.0216.00070 B2         Environmental      Water             E7727        
 6 ESCO.0216.00071 B1         Environmental      Water             E2508        
 7 ESCO.0216.00072 B2         Host-associated    Human_Intestinal  54_1_Ti6     
 8 ESCO.0216.00073 D          Host-associated    Human_Intestinal  47_4_R1      
 9 ESCO.0216.00074 B2         Host-associated    Mammal_Faecal     M714         
10 ESCO.0216.00075 B2         Environmental      Water    

In [8]:
%%R -o best_hits_assemblyID
best_hits_assemblyID <- best_hits_hmmer %>%
	separate(cinful_id,c(NA, "ASSEMBLY_ACC_ID"), sep="/") %>%
	separate(ASSEMBLY_ACC_ID, c("ASSEMBLY_ACC"), sep = "[.]")

best_hits_assemblyID

# A tibble: 769 × 25
   qseqid        sseqid pident length mismatch gapopen qstart  qend sstart  send
   <chr>         <chr>   <dbl>  <dbl>    <dbl>   <dbl>  <dbl> <dbl>  <dbl> <dbl>
 1 v1_PLS_987b1… E492_…   80.4     92       18       0      1    92      1    92
 2 v1_PLS_07196… L_tr_…   99.0    102        1       0      1   102      4   105
 3 v1_PLS_718e5… V_sp_…   81.2     16        3       0      1    16      1    16
 4 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
 5 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
 6 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
 7 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
 8 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
 9 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
10 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
# … wit

In [9]:
%%R -o best_hits_phylogroup
best_hits_phylogroup <- best_hits_assemblyID %>%
	full_join(assemblySummary_ENAInformation_phylogroup)

best_hits_phylogroup

Joining, by = "ASSEMBLY_ACC"
# A tibble: 1,676 × 33
   qseqid        sseqid pident length mismatch gapopen qstart  qend sstart  send
   <chr>         <chr>   <dbl>  <dbl>    <dbl>   <dbl>  <dbl> <dbl>  <dbl> <dbl>
 1 v1_PLS_987b1… E492_…   80.4     92       18       0      1    92      1    92
 2 v1_PLS_07196… L_tr_…   99.0    102        1       0      1   102      4   105
 3 v1_PLS_718e5… V_sp_…   81.2     16        3       0      1    16      1    16
 4 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
 5 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
 6 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
 7 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
 8 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
 9 v1_PLS_f46b0… V_sp_…  100      103        0       0      1   103      1   103
10 v1_PLS_f46b0… V_sp_…  100      103        0       0   

In [10]:
%%R -o microcin_hits_perPhylogroup
microcin_hits_perPhylogroup <- best_hits_phylogroup %>%
	filter(component == "microcins.verified") %>%
	group_by(sseqid, phylogroup) %>%
	summarise(n = n())

microcin_hits_perPhylogroup# %>%
#	write_csv("data/supplemental/microcin_hits_perPhylogroup.csv")

`summarise()` has grouped output by 'sseqid'. You can override using the
`.groups` argument.
# A tibble: 42 × 3
# Groups:   sseqid [8]
   sseqid                     phylogroup     n
   <chr>                      <chr>      <int>
 1 E492_sp_Q9Z4N4_MCEA_KLEPN  F              1
 2 H47_sp_P62530_MCHB_ECOLX   A              1
 3 H47_sp_P62530_MCHB_ECOLX   B1             5
 4 H47_sp_P62530_MCHB_ECOLX   B2            23
 5 H47_sp_P62530_MCHB_ECOLX   C              1
 6 H47_sp_P62530_MCHB_ECOLX   D              5
 7 H47_sp_P62530_MCHB_ECOLX   F              1
 8 H47_sp_P62530_MCHB_ECOLX   G              3
 9 I47_tr_Q712Q0_Q712Q0_ECOLX A              2
10 I47_tr_Q712Q0_Q712Q0_ECOLX B1             5
# … with 32 more rows


In [11]:
%%R -o microcin_hits_perPhylogroup_countMatrix
microcin_hits_perPhylogroup_countMatrix <- microcin_hits_perPhylogroup %>%
	pivot_wider(values_from=n, names_from=phylogroup)

microcin_hits_perPhylogroup_countMatrix

# A tibble: 8 × 9
# Groups:   sseqid [8]
  sseqid                         F     A    B1    B2     C     D     G     E
  <chr>                      <int> <int> <int> <int> <int> <int> <int> <int>
1 E492_sp_Q9Z4N4_MCEA_KLEPN      1    NA    NA    NA    NA    NA    NA    NA
2 H47_sp_P62530_MCHB_ECOLX       1     1     5    23     1     5     3    NA
3 I47_tr_Q712Q0_Q712Q0_ECOLX     1     2     5     5     1     6     3    NA
4 L_tr_Q841V4_Q841V4_ECOLX      NA     2     1     3    NA     1    NA    NA
5 M_tr_Q83TS1_Q83TS1_ECOLX       2     1     6    39    NA     5     3     1
6 PDI_tr_I6ZU90_I6ZU90_ECOLX     2     4     6     4     3     6    NA    NA
7 S_tr_H9ZMG7_H9ZMG7_ECOLX      NA    NA     1    NA    NA     1    NA    NA
8 V_sp_P22522_CEAV_ECOLX        12    36    37    85    10     9    17     5


So that's where the data comes from. I just took the known phylogenetic relationships of the verified micrcoins, as well as the known relationships between the phylogroups and hand drew dendrograms such that each axis is phylogenetically sorted. Then the boxes were colored by the number, and left blank f NA

Now we can do a little statistical assesment of how the phylogroups aren't equally represented in this analysis

In [12]:
%%R
n_assembliesPhylogroup <- assemblySummary_ENAInformation_phylogroup %>%
	count(phylogroup, name="n_assemblies") %>%
	mutate(freq_assemblies = n_assemblies / sum(n_assemblies))
n_assembliesPhylogroup

# A tibble: 8 × 3
  phylogroup n_assemblies freq_assemblies
  <chr>             <int>           <dbl>
1 A                   292          0.239 
2 B1                  278          0.227 
3 B2                  304          0.248 
4 C                    16          0.0131
5 D                   174          0.142 
6 E                    57          0.0466
7 F                    71          0.0580
8 G                    32          0.0261


In [13]:
%%R
best_hits_phylogroup_expected <- best_hits_phylogroup %>%
	filter(component == "microcins.verified") %>%
	group_by(phylogroup) %>%
	summarise(n_microcins = n()) %>%
	full_join(n_assembliesPhylogroup) %>%
	mutate(expected_microcins = sum(n_microcins) * freq_assemblies)
best_hits_phylogroup_expected

Joining, by = "phylogroup"
# A tibble: 8 × 5
  phylogroup n_microcins n_assemblies freq_assemblies expected_microcins
  <chr>            <int>        <int>           <dbl>              <dbl>
1 A                   46          292          0.239               87.1 
2 B1                  61          278          0.227               82.9 
3 B2                 159          304          0.248               90.7 
4 C                   15           16          0.0131               4.77
5 D                   33          174          0.142               51.9 
6 E                    6           57          0.0466              17.0 
7 F                   19           71          0.0580              21.2 
8 G                   26           32          0.0261               9.54


In [14]:
%%R
chisq.test(x = best_hits_phylogroup_expected$n_assemblies)


	Chi-squared test for given probabilities

data:  best_hits_phylogroup_expected$n_assemblies
X-squared = 702.86, df = 7, p-value < 2.2e-16



In [15]:
%%R
chisq.test(x = best_hits_phylogroup_expected$n_microcins, p=best_hits_phylogroup_expected$freq_assemblies)


	Chi-squared test for given probabilities

data:  best_hits_phylogroup_expected$n_microcins
X-squared = 141.22, df = 7, p-value < 2.2e-16



Now, a big question is in regards to the count matrix. Let's take a look at that

In [16]:
%%R
microcin_hits_perPhylogroup_countMatrix_zero <- microcin_hits_perPhylogroup_countMatrix %>% 
    ungroup() %>%
    select(sseqid,C,B1,A,E,D,F,G,B2) %>% 
    separate(sseqid, c("microcin")) %>%
    pivot_longer(C:B2, names_to = "phylogroup") %>% 
    pivot_wider(names_from = microcin, values_from = value) %>%
    select(phylogroup, H47,E492,M,I47,S,PDI,L,V) %>% # OLD ORDER -> I47,M,L,V,H47,S,PDI,E492) %>%
    pivot_longer(H47:V, names_to = "microcin") %>%
    pivot_wider(names_from = phylogroup, values_from = value) %>%
    replace(is.na(.), 0) %>%
    remove_rownames %>%
    column_to_rownames(var="microcin")

microcin_hits_perPhylogroup_countMatrix_zero

      C B1  A E D  F  G B2
H47   1  5  1 0 5  1  3 23
E492  0  0  0 0 0  1  0  0
M     0  6  1 1 5  2  3 39
I47   1  5  2 0 6  1  3  5
S     0  1  0 0 1  0  0  0
PDI   3  6  4 0 6  2  0  4
L     0  1  2 0 1  0  0  3
V    10 37 36 5 9 12 17 85


In [17]:
%%R 
microcin_hits_perPhylogroup_countMatrix_chisq <- chisq.test(microcin_hits_perPhylogroup_countMatrix_zero)

microcin_hits_perPhylogroup_countMatrix_chisq$residuals

               C         B1          A           E           D          F
H47  -0.47610015 -0.5945195 -1.7659344 -0.80068464  0.78495827 -0.7229900
E492 -0.20272121 -0.4088072 -0.3550034 -0.12821215 -0.30068415  4.1548262
M    -1.53051160 -1.1424304 -2.3071126  0.06509811 -0.06758451 -0.5614538
I47   0.05636039  0.5897082 -0.5278185 -0.61488388  2.71876903 -0.1802791
S    -0.28669109  1.1515424 -0.5020506 -0.18131937  1.92642803 -0.3226602
PDI   1.94612365  0.8913336  0.4784828 -0.64106076  2.48747798  0.6124166
L    -0.53634992 -0.1570476  1.1901064 -0.33921747  0.46147946 -0.6036419
V     0.45124089  0.2925073  1.8244573  0.82233452 -2.30710359  0.3066969
              G         B2
H47   0.1331434  1.4583418
E492 -0.2668949 -0.6600125
M    -0.5261874  2.8436497
I47   1.0637985 -1.5856837
S    -0.3774464 -0.9333986
PDI  -1.3344744 -2.0879639
L    -0.7061375 -0.0282409
V     0.5081063 -0.7212787


In [18]:
%%R 
svg("plots/06_microcins_per_phylogroup_chisq_resids.svg")
corrplot(microcin_hits_perPhylogroup_countMatrix_chisq$residuals, is.cor = FALSE, 
    # col = COL2('PiYG'), 
    method = "color")
dev.off()

png 
  2 
